In [15]:
import pandas as pd
from readii.process.subset import selectByColumnValue
from readii.process.label import setPatientIdAsIndex
from readii.io.loaders.general import loadFileToDataFrame
from pathlib import Path

In [10]:
clinical_file = Path("../data/rawdata/TCIA_RADCURE/clinical/RADCURE_Clinical_v04_20241219.xlsx")

In [11]:
raw_clinical_data = loadFileToDataFrame(clinical_file)

In [12]:
raw_clinical_data

,patient_id,Age,Sex,ECOG PS,Smoking PY,Smoking Status,Ds Site,Subsite,T,N,...,Local,Date Local,Regional,Date Regional,Distant,Date Distant,2nd Ca,Date 2nd Ca,RADCURE-challenge,ContrastEnhanced
0,RADCURE-0005,62.6,Female,ECOG 0,50,Ex-smoker,Oropharynx,post wall,T4b,N2c,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0,0
1,RADCURE-0006,87.3,Male,ECOG 2,25,Ex-smoker,Larynx,Glottis,T1b,N0,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0,1
2,RADCURE-0007,49.9,Male,ECOG 1,15,Ex-smoker,Oropharynx,Tonsil,T3,N2b,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0,1
3,RADCURE-0009,72.3,Male,ECOG 1,30,Ex-smoker,Unknown,NaN,T0,N2c,...,NaN,NaT,NaN,NaT,NaN,NaT,S (suspicious),2008-05-27,0,0
4,RADCURE-0010,59.7,Female,ECOG 0,0,Non-smoker,Oropharynx,Tonsillar Fossa,T4b,N0,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3341,RADCURE-4126,58.3,Male,ECOG 0,50,Ex-smoker,Oropharynx,Tonsil Pillar,T2,N0,...,NaN,NaT,NaN,NaT,NaN,NaT,Y,2006-11-21,training,0
3342,RADCURE-4127,52.4,Female,ECOG 0,30,Current,Oropharynx,Base of Tongue,T2,N2b,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,training,1
3343,RADCURE-4128,71.3,Male,ECOG 1,50,Ex-smoker,Oropharynx,Soft Palate,T2,N0,...,NaN,NaT,NaN,NaT,NaN,NaT,Y,2005-08-28,training,0
3344,RADCURE-4129,53.9,Female,ECOG 0,5,Current,Oropharynx,Tonsillar Fossa,T2,N2b,...,Yes,2001-12-05,NaN,NaT,NaN,NaT,NaN,NaT,0,0


In [25]:
include={'Ds Site': ["Oropharynx"]}

indexed_clinical_data = setPatientIdAsIndex(raw_clinical_data, "patient_id")
opc_clinical_data = selectByColumnValue(indexed_clinical_data, include_col_values=include)

In [26]:
opc_clinical_data.to_csv("../data/procdata/TCIA_RADCURE/clinical/OPC_only_RADCURE_Clinical_20250731.csv", index=True, index_label='patient_id')